# 8章：ニューラルネット
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ。  

## 70. 単語ベクトルの和による特徴量

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$


 ここで，$n$は学習データの事例数であり，$\boldsymbol x_i \in \mathbb{R}^d$と$y_i \in \mathbb N$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
 なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb N_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb N_{<4}$で表現できる．
 以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

 $i$番目の事例の特徴ベクトル$\boldsymbol x_i$は，次式で求める．

 $$\boldsymbol x_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})$$

 ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．  
 すなわち，**$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol x_i$である．**今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．  
 $i$番目の事例のラベル$y_i$は，次のように定義する．

$$
y_i = \begin{cases}
0 & (\mbox{記事}\boldsymbol x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}\boldsymbol x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}\boldsymbol x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}\boldsymbol x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

 + 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
 + 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
 + 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
 + 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
 + 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
 + 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

In [1]:
# データのダウンロード
# 記事の見出しとそのカテゴリが入ったデータ
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip

--2021-06-29 07:11:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  57.6MB/s    in 0.5s    

2021-06-29 07:11:37 (57.6 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [ ]:
# データの読み込み、タイトルとカテゴリの抽出、学習データ・検証データ・評価データへの適切な分割

import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('./newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# タイトルとカテゴリの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])


In [ ]:
import gdown  # Download a large file from Google Drive.
# If you use curl/wget, it fails with a large file because of the security warning from Google Drive.
from gensim.models import KeyedVectors

# 学習済み単語ベクトルのダウンロード
url = "https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM"
output = 'GoogleNews-vectors-negative300.bin.gz'
gdown.download(url, output, quiet=True)

# ダウンロードファイルのロード
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
import string
import torch

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))  # str.translate() に使える変換テーブルを返します。
  words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
  vec = [model[word] for word in words if word in model]  # 単語が単語ベクトルにあるものなら、テキスト中の単語を1語ずつベクトル化
  # 各単語の単語ベクトルを全て足し合わせ、その平均を特徴ベクトルとする
  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [ ]:
# 記事タイトルを１つづつ取り出し、特徴ベクトルの作成
X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])  # Concatenates a sequence of tensors along a new dimension.
                                                                         # All tensors need to be of the same size.
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])

print(X_train.size())  # (記事タイトル数, 単語ベクトルの次元数)
print(X_train)

In [ ]:
# ラベルベクトルの作成...記事カテゴリを正解ラベルに変換
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category_dict[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category_dict[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category_dict[x]).values)

print(y_train.size())
print(y_train)

## 71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$ 
\hat{y}_1=softmax(x_1W),\\\hat{Y}=softmax(X_{[1:4]}W)
$$


ただし，$softmax$はソフトマックス関数，$X_{[1:4]}∈\mathbb{R}^{4×d}$は特徴ベクトル$x_1$,$x_2$,$x_3$,$x_4$を縦に並べた行列である．

$$
X_{[1:4]}=\begin{pmatrix}x_1\\x_2\\x_3\\x_4\end{pmatrix}
$$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．  
なお，$\hat{\boldsymbol y_1} \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．


$d$：単語ベクトルの次元数  
$L$：記事のカテゴリ数  
$n$：記事数  
　 $Y　= 　　X 　W$  
$(n×L)　(n×d)(d×L)$  

記事のベクトルに重みを掛けて、各カテゴリに属する確率(的な値)を求める

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):  #  define our neural network by subclassing nn.Module
  def __init__(self, input_size, output_size):  # initialize the neural network layers in __init__
    super().__init__()
    self.liner1 = nn.Linear(input_size, output_size, bias=False)  # The linear layer is a module that applies a linear transformation
                                                                # using its stored weights and biases  # バイアス無くても成立はず
    nn.init.normal_(self.liner1.weight, 0.0, 1.0)  # 正規乱数で重みを初期化  # 無くても成立はするはず

    self.one_liner_stack = nn.Sequential(  # nn.Sequential is an ordered container of modules
        self.liner1                        # The input is passed through all the modules in the same order as defined
    )

  def forward(self, x):  # Every nn.Module subclass implements the forward method
    x = self.one_liner_stack(x)
    return x

In [ ]:
model = NeuralNetwork(300, 4)  # 単層ニューラルネットワークの初期化

# To use the model, we pass it the input data. This executes the model’s forward.
# Do not call model.forward() directly!
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
print(y_hat_1)

In [ ]:
Y_hat = torch.softmax(model(X_train[:4]), dim=-1)
print(Y_hat)

## 72. 損失と勾配の計算

学習データの事例x1
と事例集合x1,x2,x3,x4
に対して，クロスエントロピー損失と，行列W
に対する勾配を計算せよ．なお，ある事例xi
に対して損失は次式で計算される．

$$
l_i=−log[事例x_iがy_iに分類される確率]
$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

In [ ]:
loss_fn = nn.CrossEntropyLoss()  # This criterion combines LogSoftmax and NLLLoss in one single class.

In [ ]:
l_1 = criterion(model(X_train[:1]), y_train[:1])  # X_trainは、記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したもの
                                                  # model(X_train)はsoftmaxに通す前の(=確率にする前の)出力(カテゴリの数だけある)
                                                    # The input is expected to contain raw, unnormalized scores for each class.
                                                  # y_trainは、正解カテゴリ名を数字ラベルで表したもの(正解のカテゴリ番号１つでよい)
                                                    # ↑one_hot encodingが必要ないってこと？
model.zero_grad()  # 勾配をゼロで初期化  # ２回目以降、前の勾配の値が残ったままになるのを防ぐ
l_1.backward()  # 勾配を計算
print(f'損失: {l_1:.4f}')
print(f'勾配:\n{model.liner1.weight.grad}')

In [ ]:
l = loss_fn(model(X_train[:4]), y_train[:4])
model.zero_grad()
l.backward()
print(f'損失: {l:.4f}')
print(f'勾配:\n{model.liner1.weight.grad}')

## 73. 確率的勾配降下法による学習

確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列W
を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [ ]:
from torch.utils.data import Dataset  # Dataset stores the samples and their corresponding labels

# A custom Dataset class must implement three functions: __init__, __len__, and __getitem__
class NewsDataset(Dataset):  
  def __init__(self, X, y):  # datasetの構成要素を指定(データの初期設定)
    self.X = X
    self.y = y

  def __len__(self):  # The __len__ function returns the number of samples in our dataset
    return len(self.y)

  def __getitem__(self, idx):  # dataset[idx]で返す値を指定  # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    return [self.X[idx], self.y[idx]]

In [ ]:
# The Dataset retrieves our dataset’s features and labels one sample at a time
# While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting.
# DataLoader is an iterable that abstracts this complexity for us in an easy API
from torch.utils.data import DataLoader

# Datasetの作成
dataset_train = NewsDataset(X_train, y_train)  # 記事数×記事の特徴ベクトル, 記事カテゴリのラベル数字
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# Dataloaderの作成
# DataloaderはDatasetを入力とし、指定したサイズ(batch_size)にまとめたデータを順に取り出すことができます
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)   # データを１つずつ取り出す
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)  # データを1336(検証データの数)ずつ取り出す(?)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

# Dataloaderはfor文で順に取り出すか、またはnext(iter(Dataloader))で次のかたまりを呼び出すことが可能です。

学習の流れ(1エポック)  
* モデルの出力を計算(～softmaxの前)
* 損失を求める(～CrossEntopyLoss)
* 各パラメータについて損失に対する偏微分の値を求める(～backward)
* 勾配降下法に基づいてパラメータを最適化する(～SGD)

In [ ]:
# モデルの定義
model = NeuralNetwork(300, 4)

# ハイパーパラメータの定義
learning_rate = 1e-1
# batch_size = 64  # dataloaderがバッチを分けたデータを返してくれる
epochs = 10

# 損失関数の定義
loss_fn = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

In [ ]:
def train_loop(dataloader_train, dataloader_valid, model, loss_fn, optimizer):
    Loss_train = 0.0
    for batch, (X, y) in enumerate(dataloader_train):  # batch : 何バッチ目か。全部で10684バッチ
        # 予測と損失の計算
        pred_train = model(X)  # size(X) = (1, 300)(記事数, 記事の特徴ベクトル)
        loss_train = loss_fn(pred_train, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()  # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration
        loss_train.backward()
        optimizer.step()  #  adjust the parameters by the gradients collected in the backward pass

        # 損失を記録
        Loss_train += loss_train.item()
    # バッチ単位の平均損失計算
    Loss_train = Loss_train / batch

        # if batch % 1000 == 0:  # バッチのセットが1000回取り出されたら、出力を確認
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    # 1エポック毎に検証データの損失計算
    # model.eval() 
    with torch.no_grad():  # 検証中は伝播記録を残さない
      X, y = next(iter(dataloader_valid)) # size(X) = (300, 300)(記事数, 記事の特徴ベクトル)
      pred_valid = model(X)
      loss_valid = loss_fn(pred_valid, y)

    # １エポック毎にログを出力
    print(f'loss_train: {Loss_train:.4f}, loss_valid: {loss_valid:.4f}')



In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader_train, dataloader_valid, model, loss_fn, optimizer)    
print("Done!")

## 74. 正解率の計測

問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

学習した重みで予測をする

In [ ]:
def calculate_accuracy(model, dataloader):
    size = len(dataloader.dataset)  # 10684かな？
    total, correct = 0, 0

    with torch.no_grad():  # テスト中は伝播記録をしない
        for X, y in dataloader:
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    return correct / size

In [ ]:
acc_train = calculate_accuracy(model, dataloader_train)
acc_test = calculate_accuracy(model, dataloader_test)
print(f'正解率（学習データ）：{(100*acc_train):>0.1f}%')  # >は右詰めの記号
print(f'正解率（評価データ）：{(100*acc_test):>0.1f}%')

## 75. 損失と正解率のプロット
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

In [ ]:
def calculate_loss_and_accuracy(model, dataloader, loss_fn):
    total_loss = len(dataloader)  # バッチ数(損失はバッチ毎(1イテレータ)に1つ求まるので)
    total_data = dataloader.batch_size * len(dataloader)  # ※全データ =  (バッチ内のデータ数) * (バッチ数)
                                  # 予測結果はバッチ毎に、バッチ内のデータ数だけ求まる
    correct, loss = 0, 0

    with torch.no_grad():  # テスト中は伝播記録をしない
        for X, y in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()  # バッチ内に複数データがあるなら、各データの損失の平均を返す?
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    return loss / total_loss, correct / total_data

In [ ]:
print(dataloader_train.batch_size) # 1バッチ内のデータ数
print(len(dataloader_train))  # バッチが何個あるのか
print(dataloader_valid.batch_size) 
print(len(dataloader_valid))


In [ ]:
# モデルの定義
model = NeuralNetwork(300, 4)

# ハイパーパラメータの定義
learning_rate = 1e-1
# batch_size = 64  # dataloaderがバッチを分けたデータを返してくれる
epochs = 10

log_train = []
log_valid = []

# 損失関数の定義
loss_fn = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

In [ ]:
def train_loop(dataloader_train, dataloader_valid, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(dataloader_train):  # batch : 何バッチ目か。全部で10684バッチ
        # 予測と損失の計算
        pred_train = model(X)  # size(X) = (1, 300)(記事数, 記事の特徴ベクトル)
        loss_train = loss_fn(pred_train, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()  # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration
        loss_train.backward()
        optimizer.step()  #  adjust the parameters by the gradients collected in the backward pass

    # 各エポックのパラメータ更新が完了するたびに(全データで？)損失と正解率の算出
    loss_train, acc_train = calculate_loss_and_accuracy(model, dataloader_train, loss_fn)
    loss_valid, acc_valid = calculate_loss_and_accuracy(model, dataloader_valid, loss_fn)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    # １エポック毎にログを出力
    print(f'loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}')


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader_train, dataloader_valid, model, loss_fn, optimizer)    
print("Done!")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# 視覚化
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].plot(np.array(log_train).T[0], label='train')
ax[0].plot(np.array(log_valid).T[0], label='valid')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].legend()
ax[1].plot(np.array(log_train).T[1], label='train')
ax[1].plot(np.array(log_valid).T[1], label='valid')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('accuracy')
ax[1].legend()
plt.show()

## 76. チェックポイント

問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．



学習途中のパラメータはmodel.state_dict()、最適化アルゴリズムの内部状態はoptimizer.state_dict()でアクセス可能なので、各エポックでエポック数と合わせて保存する処理を追加

In [ ]:
def train_loop(dataloader_train, dataloader_valid, model, loss_fn, optimizer, epoch):
    for batch, (X, y) in enumerate(dataloader_train):  # batch : 何バッチ目か。全部で10684バッチ
        # 予測と損失の計算
        pred_train = model(X)  # size(X) = (1, 300)(記事数, 記事の特徴ベクトル)
        loss_train = loss_fn(pred_train, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()  # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration
        loss_train.backward()
        optimizer.step()  #  adjust the parameters by the gradients collected in the backward pass

    # 各エポックのパラメータ更新が完了するたびに(全データで？)損失と正解率の算出
    loss_train, acc_train = calculate_loss_and_accuracy(model, dataloader_train, loss_fn)
    loss_valid, acc_valid = calculate_loss_and_accuracy(model, dataloader_valid, loss_fn)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

     # チェックポイントの保存
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    # １エポック毎にログを出力
    print(f'loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}')


## 77. ミニバッチ化

問題76のコードを改変し，B
事例ごとに損失・勾配を計算し，行列W
の値を更新せよ（ミニバッチ化）．B
の値を1,2,4,8,…
と変化させながら，1エポックの学習に要する時間を比較せよ．



バッチサイズを変えるごとにすべての処理を書くのは大変なので、Dataloaderの作成以降の処理をtrain_modelとして関数化し、バッチサイズを含むいくつかのパラメータを引数として設定

In [ ]:
import time

def train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, epochs):
  # dataloaderの作成
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

  # 学習
  log_train = []
  log_valid = []
  for epoch in range(epochs):
    # 開始時刻の記録
    s_time = time.time()

    # 訓練モード
    # model.train()
    for X, y in dataloader_train:
      # 勾配をゼロで初期化
      optimizer.zero_grad()

      # 順伝播 + 誤差逆伝播 + 重み更新
      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()

    # 損失と正解率の算出
    loss_train, acc_train = calculate_loss_and_accuracy(model, dataloader_train, loss_fn)  # dataとlossの位置が参考と逆なので注意
    loss_valid, acc_valid = calculate_loss_and_accuracy(model, dataloader_valid, loss_fn)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    # チェックポイントの保存
    # torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    # 終了時刻の記録
    e_time = time.time()

    # ログを出力
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec') 

  return {'train': log_train, 'valid': log_valid}

In [ ]:
# datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)

# モデルの定義
model = NeuralNetwork(300, 4)

# 損失関数の定義
loss_fn = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# モデルの学習
for batch_size in [2 ** i for i in range(11)]:
  print(f'バッチサイズ: {batch_size}')
  log = train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, 1)

Pythonはデータをまとめて処理する方が計算時間が速くなる

## 78. GPU上での学習

問題77のコードを改変し，GPU上で学習を実行せよ．

GPUを指定する引数deviceをcalculate_loss_and_accuracy、train_modelに追加します。
それぞれの関数内で、モデルおよび入力TensorをGPUに送る処理を追加し、deviceにcudaを指定すればGPUを使用することができます。

In [ ]:
def calculate_loss_and_accuracy(model, dataloader, loss_fn, device):
    total_loss = len(dataloader)  # バッチ数(損失はバッチ毎(1イテレータ)に1つ求まるので)
    total_data = dataloader.batch_size * len(dataloader)  # ※全データ =  (バッチ内のデータ数) * (バッチ数)
                                  # 予測結果はバッチ毎に、バッチ内のデータ数だけ求まる
    correct, loss = 0, 0

    with torch.no_grad():  # テスト中は伝播記録をしない
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)           
            pred = model(X)
            loss += loss_fn(pred, y).item()  # バッチ内に複数データがあるなら、各データの損失の平均を返す?
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    return loss / total_loss, correct / total_data

In [ ]:
import time
def train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, epochs, device=None):

  # GPUに送る
  model.to(device)

  # dataloaderの作成
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

  # 学習
  log_train = []
  log_valid = []
  for epoch in range(epochs):
    # 開始時刻の記録
    s_time = time.time()

    # 訓練モード
    # model.train()
    for X, y in dataloader_train:
      # 勾配をゼロで初期化
      optimizer.zero_grad()

      # 順伝播 + 誤差逆伝播 + 重み更新
      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()

    # 損失と正解率の算出
    loss_train, acc_train = calculate_loss_and_accuracy(model, dataloader_train, loss_fn, device)  # dataとlossの位置が参考と逆なので注意
    loss_valid, acc_valid = calculate_loss_and_accuracy(model, dataloader_valid, loss_fn, device)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    # チェックポイントの保存
    # torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    # 終了時刻の記録
    e_time = time.time()

    # ログを出力
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec') 

  return {'train': log_train, 'valid': log_valid}

In [ ]:
# datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)

# モデルの定義
model = NeuralNetwork(300, 4)

# 損失関数の定義
loss_fn = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# デバイスの指定
device = torch.device('cuda')

# モデルの学習
for batch_size in [2 ** i for i in range(11)]:
  print(f'バッチサイズ: {batch_size}')
  log = train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, 1)

## 79. 多層ニューラルネットワーク

問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ

In [ ]:
from torch import nn

class DeepNN(nn.Module):  #  define our neural network by subclassing nn.Module
  def __init__(self, input_size, output_size):  # initialize the neural network layers in __init__
    super().__init__()
    self.liner_stack = nn.Sequential(
            nn.Linear(input_size,200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, output_size),
            nn.ReLU()
        )

  def forward(self, x):  # Every nn.Module subclass implements the forward method
    x = self.liner_stack(x)
    return x

In [ ]:
def calculate_loss_and_accuracy(model, dataloader, loss_fn, device):
    total_loss = len(dataloader)  # バッチ数(損失はバッチ毎(1イテレータ)に1つ求まるので)
    total_data = dataloader.batch_size * len(dataloader)  # ※全データ =  (バッチ内のデータ数) * (バッチ数)
                                  # 予測結果はバッチ毎に、バッチ内のデータ数だけ求まる
    correct, loss = 0, 0

    with torch.no_grad():  # テスト中は伝播記録をしない
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)           
            pred = model(X)
            loss += loss_fn(pred, y).item()  # バッチ内に複数データがあるなら、各データの損失の平均を返す?
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    return loss / total_loss, correct / total_data

In [ ]:
import time
def train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, epochs, device=None):

  # GPUに送る
  model.to(device)

  # dataloaderの作成
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

  # 学習
  log_train = []
  log_valid = []
  for epoch in range(epochs):
    # 開始時刻の記録
    s_time = time.time()

    # 訓練モード
    # model.train()
    for X, y in dataloader_train:
      # 勾配をゼロで初期化
      optimizer.zero_grad()

      # 順伝播 + 誤差逆伝播 + 重み更新
      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()

    # 損失と正解率の算出
    loss_train, acc_train = calculate_loss_and_accuracy(model, dataloader_train, loss_fn, device)  # dataとlossの位置が参考と逆なので注意
    loss_valid, acc_valid = calculate_loss_and_accuracy(model, dataloader_valid, loss_fn, device)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])

    # チェックポイントの保存
    # torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

    # 終了時刻の記録
    e_time = time.time()

    # ログを出力
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec') 

    
    # 検証データの損失が3エポック連続で低下しなかった場合は学習終了
    if epoch > 2 and log_valid[epoch - 3][0] <= log_valid[epoch - 2][0] <= log_valid[epoch - 1][0] <= log_valid[epoch][0]:
      break
    
  return {'train': log_train, 'valid': log_valid}

In [ ]:
# datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)

# モデルの定義
model = DeepNN(300, 4)

# 損失関数の定義
loss_fn = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# デバイスの指定
device = torch.device('cuda')

# モデルの学習
batch_size = 64
log = train_model(dataset_train, dataset_valid, batch_size, model, loss_fn, optimizer, 100)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].plot(np.array(log['train']).T[0], label='train')
ax[0].plot(np.array(log['valid']).T[0], label='valid')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].legend()
ax[1].plot(np.array(log['train']).T[1], label='train')
ax[1].plot(np.array(log['valid']).T[1], label='valid')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('accuracy')
ax[1].legend()
plt.show()